## FDW Crop Production Data Profiling - Chad

In [1]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('./data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32633' # Chad
fn_shapes = sorted(glob.glob('./data/shapefile/fewsnet/TD_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Chad',
    'product': 'R011',
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df)
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([TD_Admin1_1960, TD_Admin1_2008, TD_Admin1_2012], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
        Admin1  # units    Admin2  # units
year                                      
1960  TD1960A1       14  TD1960A2       52
2008  TD2008A1       22  TD2008A2       62
2012  TD2012A1       23  TD2012A2       68
2017  TD2017A1       23  TD2017A2       70
----------------------------------------------- #
- Remove missing records ---------------------- #
Orignial data points: 5,522
Removed 2 "Collected" points
Removed 1 "Missing Historic Data" points
Removed 24 "Not Collected" points
1,786/1,787 "Area Harvested" points are retained.
1,775/1,787 "Quantity Produced" points are retained.
1,934/1,948 "Yield" points are retained.
Current data points: 5,495

- Minor changes are applied.. ----------------- #

- Basic information --------------------------- #
Data period: 1983 - 2017
6 grain types are found: Fonio, Maize (Corn), Millet, Rice (Paddy), Sorghum, Wheat Grain
2 seasons are found: Main harvest (10-01), Cold off-season harvest (02-01


- Chad crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large/public/seasonal-calendar-chad.png?itok=PUNJhiRe)

- FDW data consists of `TD1960A1`, `TD2008A1`, and `TD2012A1`.

| Year | Admin-1 | # units  | Admin-2  | # units |
| :---: | :----:  | :----:   | :----:   | :---:  |
| 1960 | TD1960A1| 14        | TD1960A2 | 52     |
| 2008 | TD2008A1| 22        | TD2008A2 | 62     |
| 2012 | **`TD2012A1`**| 23 | TD2012A2 | 68    |

- Comparison between admin boundaries.

![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_admin_shapes.png?raw=true)

- In 2008, 8 districts are divided and added.

| 1960-2008 |2008-2012|
| :---:|:---:|
|TD1960A124 (Chari-Baguirmi) | TD2008A104 (Chari-Baguirmi), TD2008A108 (Hadjer-Lamis), TD2008A117 (N'Djamena) |
|TD1960A125 (Kanem) | TD2008A109 (Kanem), TD2008A101 (Barh el Gazel)|
|TD1960A126 (Moyen-Chari)| TD2008A116 (Moyen-Chari), TD2008A113 (Mandoul)|
|TD1960A127 (Ouaddai) | TD2008A118 (Ouaddai), TD2008A120 (Sila) | 
|TD1960A128 (B.E.T.) | TD2008A122 (Tibesti), TD2008A103 (Borkou), TD2008A130 (Ennedi)|
|TD1960A129 (Mayo-Kebbi) | TD2008A114 (Mayo-Kebbi Est), TD2008A115 (Mayo-Kebbi Ouest) |

- In 2012, 1 district is divided and added.

| 2008-2012| 2012-present |
| :---:|:---:|
|TD2008A130 (Ennedi) | TD2012A105 (Ennedi-Est), TD2012A106 (Ennedi-Ouest)|

- **`TD2012A1`** is used to represent current admin-level 1 crop data.
- Chad has two crop seasons: `Main harvest` and `Cold off-season`.
- Chad has no population group(s).

In [3]:
# Define the latest shapefile ------------------- #
latest_level = 1
shape_latest = TD_Admin1_2012.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
df['season_name'] = df['season_name'].replace({'Main harvest':'Main', 'Cold off-season harvest': 'Cold-off'})
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

In [4]:
# Link admin boundaries ------------------------- #
link_1960, over_1960 = FDW_PD_CreateAdminLink(TD_Admin1_1960, TD_Admin1_2012, 'ADMIN1', 'ADMIN1', area, epsg)
link_1960['TD1960A128'].update(method='ABR')
link_1960['TD1960A124'].update(method='ABR')
assert all(np.unique([v['method'] for k,v in link_1960.items()]) == 'ABR')
link_2008, over_2008 = FDW_PD_CreateAdminLink(TD_Admin1_2008, TD_Admin1_2012, 'ADMIN1', 'ADMIN1', area, epsg)
link_2008['TD2008A130'].update(method='ABR')
link_2008['TD2008A122'].update(method='ABR')
link_2008['TD2008A103'].update(method='ABR')
link_2008['TD2008A108'].update(method='ABR')
link_2008['TD2008A117'].update(method='ABR')
assert all(np.unique([v['method'] for k,v in link_2008.items()]) == 'ABR')
# Crop specific ratios
link_ratio_1960 = FDW_PD_RatioAdminLink(link_1960, area, over_1960, mdx_pss)
link_ratio_2008 = FDW_PD_RatioAdminLink(link_2008, area, over_2008, mdx_pss)
# Merge link_ratio
assert link_ratio_1960.keys() == link_ratio_2008.keys()
link_merged = [link_ratio_1960, link_ratio_2008]
fnids_new = list(link_merged[0].keys())
link_ratio = dict()
for fnid in fnids_new:
    container = []
    for link in link_merged:
        container.append(link[fnid])
    link_ratio[fnid] = pd.concat(container, axis=1)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=True)
# ----------------------------------------------- #

# Aggregate grain data by grain type ------------ #
[area_new, prod_new, area_all, prod_all] = FDW_PD_GrainTypeAgg([area_new, prod_new, area_all, prod_all], product_category)
# ----------------------------------------------- #

# Manual correction ----------------------------- #
crop_new = prod_new/area_new
# ----------------------------------------------- #

# Complete long format DataFrame
df_area = area_new.T.stack().reset_index().rename({0:'value'},axis=1)
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().reset_index().rename({0:'value'},axis=1)
df_prod['indicator'] = 'production'
df_yield = (prod_new/area_new).T.stack().reset_index().rename({0:'value'},axis=1)
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Insert a country name
stack['country'] = 'Chad'
stack = stack[['fnid','country','name','product','year','season_name','season_date','indicator','value']]
stack = stack.reset_index(drop=True)
# Change season_date to harvest_end
stack.rename(columns={'season_date':'harvest_end'},inplace=True)
stack['harvest_end'] = stack['harvest_end'].replace({
    '10-01':'11-01', # Main
    '02-01':'03-01', # Cold off-season
})

# Save data
save_hdf('./data/crop/adm_crop_production_raw_TD.hdf', df)
save_hdf('./data/crop/adm_crop_production_TD.hdf', stack)
save_npz('./data/crop/adm_crop_production_TD_ratio.npz', link_ratio)

CBR is considered for 'TD1960A128' as no record found in: ['TD2012A103', 'TD2012A105', 'TD2012A106', 'TD2012A122']
CBR is considered for 'TD1960A124' as no record found in: ['TD2012A117']
CBR is considered for 'TD2008A130' as no record found in: ['TD2012A105', 'TD2012A106']
CBR is considered for 'TD2008A122' as no record found in: ['TD2012A122']
CBR is considered for 'TD2008A103' as no record found in: ['TD2012A103']
- Aggregation of grain types ------------------ #
6 crops: Fonio, Maize (Corn), Millet, Rice (Paddy), Sorghum, Wheat Grain
6 crops: Fonio, Maize, Millet, Rice, Sorghum, Wheat

./data/crop/adm_crop_production_raw_TD.hdf is saved.
./data/crop/adm_crop_production_TD.hdf is saved.
./data/crop/adm_crop_production_TD_ratio.npz is saved.


## Visualization of production data

In [5]:
# Bar chart of national grain production
country_iso, country_name = 'TD', 'Chad'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
year = [df['year'].min(), df['year'].max()]
product_order = ['Sorghum','Millet','Maize','Rice','Fonio','Wheat']
for season_name in ['Main','Cold-off']:
    footnote = 'National grain production in %s - %s' % (country_name, season_name)
    fn_save = './figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    # fig.show()

./figures/TD_bar_natgrainprod_Main.png is saved.
./figures/TD_bar_natgrainprod_Cold-off.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_bar_natgrainprod_Main.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_bar_natgrainprod_Cold-off.png?raw=true)

In [6]:
# Lineplot of Production-Area-Yield (PAY) time-series
country_iso, country_name = 'TD', 'Chad'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Sorghum','Main'],
    ['Millet','Main'],
    ['Maize','Main'],
    ['Sorghum','Cold-off'],
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield (PAY) time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = './figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    # fig.show()

./figures/TD_line_pay_Sorghum_Main.png is saved.
./figures/TD_line_pay_Millet_Main.png is saved.
./figures/TD_line_pay_Maize_Main.png is saved.
./figures/TD_line_pay_Sorghum_Cold-off.png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_line_pay_Sorghum_Main.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_line_pay_Millet_Main.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_line_pay_Maize_Main.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_line_pay_Sorghum_Cold-off.png?raw=true)

In [7]:
# Heatmap of seasonal data availability
country_iso, country_name = 'TD', 'Chad'
df = pd.read_hdf('./data/crop/adm_crop_production_raw_%s.hdf' % country_iso)
code = {'Main':1, 'Cold-off':2}
comb = {1:1,10:2,11:3}
comb_name = {1:'Main',2:'Cold-off',3:'Main + Cold-off'}
for product_name in ['Sorghum','Millet','Maize (Corn)']:
    data = df[(df['product'] == product_name) & (df['season_name'].isin(code.keys()))]
    footnote = 'Seasonal data availability in %s - %s (uncorrected)' % (country_name, product_name)
    fn_save = './figures/%s_heat_seasondata_%s.png' % (country_iso, product_name)
    fig = PlotHeatSeasonData(data, code, comb, comb_name, footnote, fn_save)
    # fig.show()

./figures/TD_heat_seasondata_Sorghum.png is saved.
./figures/TD_heat_seasondata_Millet.png is saved.
./figures/TD_heat_seasondata_Maize (Corn).png is saved.


![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_heat_seasondata_Sorghum.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_heat_seasondata_Millet.png?raw=true)
![image](https://github.com/chc-ucsb/gscd/blob/main/figures/TD_heat_seasondata_Maize%20(Corn).png?raw=true)

In [9]:
# Calibrated PAY time-series per FNID
from tools_graphic import PlotLineCropTS
country_iso, country_name = 'TD', 'Chad'
df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % country_iso)
link_ratio = load_npz('./data/crop/adm_crop_production_%s_ratio.npz' % country_iso)
year_all = np.arange(df['year'].min(), df['year'].max()+1)
prod_season = [
    ['Sorghum','Main'],
    ['Millet','Main'],
    ['Maize','Main'],
    ['Sorghum','Cold-off'],    
]
for product_name, season_name in prod_season:
    sub = df[
        (df['product'] == product_name) &
        (df['season_name'] == season_name)
    ]
    for fnid in sub['fnid'].unique():
        sub_fps = sub[sub['fnid'] == fnid]
        fn_save = './figures/crop_calibrated/%s_%s_%s_%s.png' % (country_iso, product_name, season_name, fnid)
        # fig = PlotLineCropTS(sub_fps, fnid, product_name, season_name, link_ratio, year_all, fn_save)